In [2]:
from backends import ModelSpec, Model, get_model_for, load_model_registry


In [3]:
load_model_registry() # load default model registry from backends folder


In [4]:
THIS_MODEL = 'gpt-3.5-turbo-1106' # fails if no registered entry


In [5]:
#need to add api key
#need to add model_name in the dict
#need to create key.json
THIS_MODEL = dict(model_id="gpt-3.5-turbo-1106", backend="openai", model_name = "gpt-3.5-turbo-1106") # works without registered entry when openai_api.py backend is available


In [5]:
lmm: Model = get_model_for(THIS_MODEL)


In [10]:
lmm.set_gen_args(temperature = 0.0, max_tokens= 100) 


In [11]:
?lmm.generate_response


In [12]:
def add_message(context, msg, role='user', sysmsg = 'You are a helpful assistant'):
    if context == []:
        context = [
            {"role": "system", "content": sysmsg}
        ]
    context.append({"role": role, "content": msg})
    return context


In [15]:
add_message([], "hello, how are you?")


In [18]:
prompt = add_message([], "What is benchmarking?")
_, resp, resp_text = lmm.generate_response(prompt)
resp_text


In [20]:
from string import Template

init_prompt_A = Template('''Let us play a game. I will tell you a topic and you will give me a short sentence that fits to this topic.
But I will also tell you a letter. The sentence that you give me has to start with this letter.
After you have answered, I will give you my reply, which will start with the letter following your letter in the alphabet.
Then it is your turn again, to produce a reply starting with the letter following that one. And so on. Let's go.
Start your utterance with I SAY: and do no produce any other text.
The topic is: $topic
The letter is: $letter''')

In [21]:
topic = 'birds'
letter = 'h'
init_prompt_A.substitute(topic=topic, letter=letter)


In [22]:
prompt_A, resp, resp_text = lmm.generate_response(add_message([], init_prompt_A.substitute(topic='birds', letter='h')))
resp_text


In [23]:
prefix = 'I SAY: '
def parse_reply(text, prefix=prefix):
    if not text.startswith(prefix):
        return False
    return text[len(prefix):]

In [24]:
parse_reply(resp_text)


In [25]:
def check_move(text, letter):
    token = text.split()
    if token[0][0].lower() == letter: # and token[-1][0].lower() == letter:
        return True
    return False

In [26]:
check_move(parse_reply(resp_text), letter)


## Testing for Ludo now


In [11]:
from backends import ModelSpec, Model, get_model_for, load_model_registry

load_model_registry()  # load default model registry from backends folder


In [12]:
THIS_MODEL = dict(model_id="gpt-3.5-turbo-1106", backend="openai", model_name = "gpt-3.5-turbo-1106") # works without registered entry when openai_api.py backend is available


In [14]:
from string import Template

# init_prompt_A = Template('''Let us play a game. I will tell you a topic and you will give me a short sentence that fits to this topic.
# But I will also tell you a letter. The sentence that you give me has to start with this letter.
# After you have answered, I will give you my reply, which will start with the letter following your letter in the alphabet.
# Then it is your turn again, to produce a reply starting with the letter following that one. And so on. Let's go.
# Start your utterance with I SAY: and do no produce any other text.
# The topic is: $topic
# The letter is: $letter''')

In [258]:
with open('resources/initial_prompts/simple_prompt_v1.txt', 'r') as f:
    content = f.read()



In [52]:
text_prompt = ""

def add_message(context, msg, role='user', sysmsg = content):
    if context == []:
        context = [
            {"role": "system", "content": sysmsg}
        ]
    context.append({"role": role, "content": msg})
    return context


In [53]:
in_m = "[start] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [][end]\n Roll: 6"
prompt = add_message([], in_m)


In [54]:
_, resp, resp_text = lmm.generate_response(prompt)


In [55]:
resp_text

In [56]:
prefix = "MY MOVE:\n"
def parse_reply(text, prefix=prefix):
    if not text.startswith(prefix):
        return False
    return text[len(prefix):]


In [57]:
parsed = parse_reply(resp_text)


In [58]:
prompt_2 = add_message(_, resp_text, role = 'assistant')
next_prompt_2 = add_message(prompt_2, 'Roll: 3', role = 'user')

next_prompt_2

In [59]:
prompt_A, resp, resp_text = lmm.generate_response(next_prompt_2)
resp_text

In [63]:
parsed = parse_reply(resp_text)

In [ ]:
token_flag = "[A:X]"
field = "[]"

def check_move(current_state, response, roll):
    current_state_list = current_state.split() #parse the fields into individual elements
    new_state_list = response.split
    
    current_state_token_inx = current_state_list.index('[A:X]')
    new_state_token_inx = new_state_list.index('[A:X]')
    
    if current_state_token_inx + roll == new_state_token_inx:
        return True
    
    else:
        return False
    
    
    
    

In [64]:
parsed

In [65]:
t = parsed.split()

In [67]:
t.index('[A:X]')

In [68]:
def generate_board_simple(n):
    
    board = "[start]"
    
    for i in range(n):
        board += " []"
    board += " [end]"
    
    return board
    
    

In [71]:
b = generate_board_simple(25)

In [72]:
b

## Try 2


In [44]:
import re

#system prompt
with open('resources/initial_prompts/simple_prompt_v1.txt', 'r') as f:
    sys_prompt = f.read()

#initial instruction
with open('resources/initial_prompts/simple_prompt_v2.txt', 'r') as f:
    content = f.read()

prefix = "MY MOVE:"
def parse_reply(text, prefix=prefix):
    parsed = text.split(prefix)
    return parsed[1]
    
    #if not text.startswith(prefix):
    #    return False
    
    #return text[len(prefix):]

def generate_board_simple(n):
    
    board = "[start]"
    
    for i in range(n):
        board += " []"
    board += " [end]"
    
    return board

def check_move(current_state, response, roll):
    current_state_list = current_state.split() #parse the fields into individual elements
    new_state_list = response.split()
    
    current_state_token_inx = current_state_list.index('[A:X]')
    new_state_token_inx = new_state_list.index('[A:X]')
    
    if current_state_token_inx + roll == new_state_token_inx:
        return True
    
    else:
        return False
    
    
def add_message(context, msg, role='user', sysmsg = sys_prompt):
    if context == []:
        context = [
            {"role": "system", "content": sysmsg}
        ]
    context.append({"role": role, "content": msg})
    return context


#game class
class Game():
    def __init__(self, lmm, initial_msg, sys_prompt):
        self.msg = initial_msg #initial message -> contains game instructions
        self.sys_prompt = sys_prompt #system prompt
        self.lmm = lmm #language model object
        self.rolls = [3,6,4,5,2,6,4,1] #list of rolls per turn
        self.board = "□ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □"
        self.turn = 0 #turn counter
        self.context = [] #memorized context
        self.current_state = '' #current state of the board
        self.prefix = "MY MOVE:" #used to parse the response
        self.token_out = False #false if no token on board yet.

    #makes move
    def make_move(self):
        if self.turn == 0: #first turn prompt is slightly different
            in_m = f"Are you ready to play?\nBeginning State: {self.board}\n Turn num: {self.turn}, Roll: {self.rolls[self.turn]}. Where will you move your token? Let's think step by step."
            self.add_message(self.msg + '\n' + in_m)
            init_prompt, init_resp, init_resp_text = self.lmm.generate_response(self.context)
            
            #needs try catch shit here, and everywhere we actually parse/split/index/etc.
            parsed = self.parse_reply(init_resp_text)
            
            #check validity of the move
            if self.check_move(parsed, self.rolls[self.turn]) == True:
                print(self.current_state)
                print(parsed)
                self.current_state = parsed 
                self.add_message(init_resp_text, role = 'assistant' ) #if valid, adds response to context
                self.turn+=1 #updates step
                if 'X' in self.current_state: #if token has been placed on the board
                    self.token_out = True

                
            else:
                print(f'fail at turn: {self.turn}')
                print(f'roll: {self.rolls[self.turn]}')
                print(f'{init_resp_text}')
        else:
            #if not first turn
            m = f"Next turn number: {self.turn}, Roll: {self.rolls[self.turn]}.\n Where will you move your token? Let's think step by step."
            self.add_message(m, role = 'user') 
            given_prompt, response, response_text = lmm.generate_response(self.context)
            
            parsed = self.parse_reply(response_text)
            if  self.check_move(parsed, self.rolls[self.turn]) == True:
                print(self.current_state)
                print(parsed)
                self.current_state = parsed
                self.add_message(response_text, role ='assistant')
                self.turn+=1
                if 'X' in self.current_state: #if token has been placed on the board
                    self.token_out = True

            else: 
                print(f'fail at turn: {self.turn}')
                print(f'roll: {self.rolls[self.turn]}')

                print(f'{response_text}')
            
    
    def add_message(self, message, role='user'):
        #add message to context
        if self.context == []:
            self.context = [
                {"role": "system", "content": self.sys_prompt}
            ]
        self.context.append({"role": role, "content": message})
    
    
    
    def check_move(self, response, roll):
        #move checker, checks validity
        if self.token_out == False:     #if the token hasn't been added to the board
            if roll != 6: 
                #can't add token if 6 isn't roleld
                for item in response.split():
                    if 'X' in item:
                        return False
                    else:
                        return True
            if roll == 6:
                #has to add token if 6 is rolled
                if 'X' in response.split()[0]:
                    return True
                else:
                    return False
        else:
            #counts the number of steps moved. Looks at the difference between the current state of the board and LLM's choice of movement. 
            current_state_list = self.current_state.split() #parse the fields into individual elements
            new_state_list = response.split()
            if len(current_state_list) != len(new_state_list):
                print('field count err')
                return False
            else:
            
                if self.token_out == True:
                    current_state_token_inx = current_state_list.index('X') + 1
                    if 'X' in response:
                        new_state_token_inx = new_state_list.index('X') + 1
                        print(current_state_token_inx, roll, new_state_token_inx)
                        if current_state_token_inx + roll == new_state_token_inx:
                            return True
                        else:
                            return False
                    else:
                        return False                                        
    def reprompt(self, text, err_resp):
        
        m = f"{text}\nThis is the current board state: {self.current_state}\nNext turn number: {self.turn}, Roll: {self.rolls[self.turn]}.\n Where will you move your token? Let's think step by step."
        self.add_message(err_resp, role='assistant')
        self.add_message(m, role = 'user') 
        given_prompt, response, response_text = lmm.generate_response(self.context)
        
        parsed = self.parse_reply(response_text)
        if  self.check_move(parsed, self.rolls[self.turn]) == True:
            print(self.current_state)
            print(parsed)
            self.current_state = parsed
            self.add_message(response_text, role ='assistant')
            self.turn+=1
            if 'X' in self.current_state: #if token has been placed on the board
                self.token_out = True
        else: 
            print(f'fail at turn: {self.turn}')
            print(f'roll: {self.rolls[self.turn]}')
            print(f'{response_text}')
            
          
    
    def parse_reply(self, text):
        #parses the reply so we only take out the board
        parsed = text.split(self.prefix)
        return parsed[1]
    
    def split(self, text):   
        #regex to split the board into a list
        parts = re.findall(r'\[.*?\]', text)
        return parts
    
    def retry(self):
        given_prompt, response, response_text = lmm.generate_response(self.context)
            
        parsed = self.parse_reply(response_text)
        if  self.check_move(parsed, self.rolls[self.turn]) == True:
            print(self.current_state)
            print(parsed)
            self.current_state = parsed
            self.add_message(response_text, role ='assistant')
            self.turn+=1
            if 'X' in self.current_state: #if token has been placed on the board
                self.token_out = True
        else: 
            print(f'fail at turn: {self.turn}')
            print(f'roll: {self.rolls[self.turn]}')
            print(f'{response_text}')
            
    
    def reset(self):
        #resets game state
        self.turn = 0
        self.context = []

In [45]:
instance = Game(lmm, content, sys_prompt)

In [52]:
instance.make_move()

In [53]:
err_m = "My token is on field number 18. You have rolled 4. Since 18 + 4 = 22, I need to move my token to field number 22. This is 4 fields away from my previous position. Since there are 23 fields on the board, I will print 21*□, then 1*X, then 1*□.\nMY MOVE: □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ X □"
new_m = "Hey! it seems that you have made a mistake in counting the number of fields. You printed more than we needed. Let's go one step back and try again."

instance.reprompt(new_m, err_m)

In [43]:
instance.context

In [31]:
instance.retry()

In [387]:
instance.current_state.split().index('X')

In [401]:
len('□ □ □ □ X □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □ □'.split())

In [7]:
lmm: Model = get_model_for(THIS_MODEL)

lmm.set_gen_args(temperature=0.0, max_tokens=400)

#?lmm.generate_response

In [380]:
in_m = "Are you ready to play?\nBeginning State: [start] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [][end]\n Turn num: 0, Roll: 6. Where will you move your token? Let's think step by step."
prompt = add_message([], content + '\n' + in_m)



In [381]:
init_prompt, init_resp, init_resp_text = lmm.generate_response(prompt)


In [382]:
init_resp_text

In [383]:
second_prompt_gpt = add_message(init_prompt, init_resp_text, role = 'assistant')
second_prompt_usr = add_message(second_prompt_gpt, "Next turn number: 1, Roll: 2.\n Where will you move your token? Let's think step by step. ", role = 'user')

In [384]:
second_prompt_resp, second_resp, second_resp_text = lmm.generate_response(second_prompt_usr)

In [385]:
second_resp_text

In [386]:
third_prompt_gpt = add_message(second_prompt_resp, second_resp_text, role='assistant')
third_prompt_usr = add_message(third_prompt_gpt, "Correct! Next turn number: 2, Roll: 5.\n Where will you move your token? Let's think step by step. ", role='user')


In [387]:
third_prompt_resp, third_resp, third_resp_text = lmm.generate_response(third_prompt_usr)


In [399]:
parsed = parse_reply(third_resp_text, prefix)
parsed_prev = parse_reply(second_resp_text, prefix)

In [398]:
third_resp_text.split(prefix)

In [391]:
third_resp

In [400]:
check_move(parsed_prev, parsed, 5)

In [88]:
parsed.split().index('[A:X]')

In [ ]:
class GameState():
    def __init__(self, board):
        self.board = board
        self.n_moves = 0
        
        

In [159]:
content

In [410]:
for item in parsed.split():
    if 'A:X' in item:
        print('y')

In [404]:
parsed

In [408]:
parsed.split()

In [25]:
import re

init_resp_text = "[start A:X] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [] [][end]"

parts = re.findall(r'\[.*?\]', init_resp_text)


In [27]:
parts